In [95]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [96]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.atl_events_db
collection = db.events

In [97]:
#set up chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\steve\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


In [98]:
#visit url
url = "https://terminalwestatl.com/calendar"
request = requests.get(url)
browser.visit(url)

In [99]:
#wait for browser to load calendar into html
time.sleep(5)

In [100]:
#parse HTML
html = browser.html
soup = bs(html, "html.parser")
browser.quit()

In [101]:
#make lists for dataframe
event_name = []
event_desc = []
event_date = []
event_time = []
event_link = []

In [102]:
#make list of all the event cards
results = soup.find_all('div', class_='c-axs-event-card__wrapper')

In [103]:
#loop through event cards
for result in results:

    #find Headlining artist or event title
    title = result.find('div', class_='c-axs-event-card__title c-axs-event-card__title7124').text
    #find opening artists and make an event description
    try:
        supporting = result.find('div', class_="c-axs-event-card__supporting-text c-axs-event-card__supporting-text7124").text
        if len(supporting) > 0:
            description = f'{title} performing with {supporting}'
    except:
        supporting = ""
        description = "n/a"
    #find date/time, then split to make a column for each
    try:
        date_time = result.find('p', class_='c-axs-event-card__date--showtime7124').text
        if date_time == "TBD":
            date = "TBD"
            time = 'TBD'
        else:
            date = date_time.split("2021")[0].strip()
            time = date_time.split("2021")[1].strip()
            date = f'{date} 2021'
    except:
        date = "TBD"
        time = 'TBD'
    #find link
    url = result.find('a', class_='c-axs-event-card__header c-axs-event-card__header7124')['href']
    link = f'https://terminalwestatl.com{url}'

    event_name.append(title)
    event_desc.append(description)
    event_date.append(date)
    event_time.append(time)
    event_link.append(link)

    event = {
        "Event_name":title,
        "Event_description":description,
        "Event_date":date,
        "Event_time":time,
        "Event_link":link
    }
    
    collection.insert_one(event)


In [104]:
#make dataframe
df = pd.DataFrame({
    "Event_name":event_name,
    "Event_description":event_desc,
    "Event_date":event_date,
    "Event_time":event_time,
    "Event_link":event_link
})

,Event_name,Event_description,Event_date,Event_time,Event_link
0,Read Southall Band,Read Southall Band performing with Midnight Ri...,"Wed, Sep 8, 2021",8:00 PM,https://terminalwestatl.com/events/detail?even...
1,Town Mountain,Town Mountain performing with Smith & Calvert,"Thu, Sep 9, 2021",8:00 PM,https://terminalwestatl.com/events/detail?even...
2,Marvel Years & Maddy O'Neal,Marvel Years & Maddy O'Neal performing with Pu...,"Fri, Sep 10, 2021",9:00 PM,https://terminalwestatl.com/events/detail?even...
3,Mark Farina,Mark Farina performing with Caleb Wheels,"Sat, Sep 11, 2021",9:30 PM,https://terminalwestatl.com/events/detail?even...
4,Emily King,n/a,"Mon, Sep 13, 2021",8:00 PM,https://terminalwestatl.com/events/detail?even...


In [106]:
df.head(5).to_csv("terminal_west.csv")